In [1]:
import torch
from utils.experimentorUtils import getExperimentor
from utils.constants import *
print(torch.backends.cudnn.version())

8005


In [2]:
config = {
    "dataset_name": Dataset.CORA,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 500,
    'num_of_runs': 1,
    'patience_period': 100,
    
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 10,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.001,
    'num_of_layers': 3, 
    'num_heads': 1,
    'hidden_size': 128,
    'dropout': 0.5,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    
    'nbor_degree': 2,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': True
}

In [3]:
experimentor = getExperimentor(config["dataset_name"])(config)
experimentor.run_wrapper()

Run 01:


Epoch 01:   5%|▌         | 140/2708 [00:04<01:19, 32.37it/s]


Epoch 01| Loss: 2.0022| Acc: 0.0096| Train Time: 4.3289s
Epoch 02| Loss: 1.8415| Acc: 0.0174| Train Time: 3.7161s
Epoch 03| Loss: 1.7222| Acc: 0.0244| Train Time: 3.3191s
Epoch 04| Loss: 1.6619| Acc: 0.0303| Train Time: 2.4143s
Epoch 05| Loss: 1.5840| Acc: 0.0347| Train Time: 2.8264s
Epoch 06| Loss: 1.4458| Acc: 0.0366| Train Time: 2.6830s
Epoch 07| Loss: 1.2843| Acc: 0.0395| Train Time: 2.6376s
Epoch 08| Loss: 1.1845| Acc: 0.0421| Train Time: 2.3691s
Epoch 09| Loss: 1.1348| Acc: 0.0410| Train Time: 2.3108s
Epoch 10| Loss: 1.0542| Acc: 0.0428| Train Time: 2.6770s


Evaluating: 100%|██████████| 8124/8124 [00:12<00:00, 668.63it/s]


Train: 0.9500| Val: 0.7660| Test: 0.7920| Eval Time: 12.1562s
Epoch 11| Loss: 0.9769| Acc: 0.0454| Train Time: 2.4618s
Epoch 12| Loss: 0.8443| Acc: 0.0476| Train Time: 2.3377s
Epoch 13| Loss: 0.8270| Acc: 0.0473| Train Time: 2.7337s
Epoch 14| Loss: 0.7330| Acc: 0.0491| Train Time: 2.4352s
Epoch 15| Loss: 0.6256| Acc: 0.0484| Train Time: 2.5193s
Epoch 16| Loss: 0.6328| Acc: 0.0469| Train Time: 2.5154s
Epoch 17| Loss: 0.5127| Acc: 0.0480| Train Time: 2.5213s
Epoch 18| Loss: 0.4719| Acc: 0.0502| Train Time: 2.4215s
Epoch 19| Loss: 0.5818| Acc: 0.0480| Train Time: 2.3780s
Epoch 20| Loss: 0.4641| Acc: 0.0495| Train Time: 2.3449s


...

Train: 0.9929| Val: 0.7720| Test: 0.7860| Eval Time: 7.1639s
Epoch 51| Loss: 0.0452| Acc: 0.0517| Train Time: 2.2621s
Epoch 52| Loss: 0.0803| Acc: 0.0517| Train Time: 2.2552s
Epoch 53| Loss: 0.0519| Acc: 0.0517| Train Time: 3.1881s
Epoch 54| Loss: 0.0615| Acc: 0.0517| Train Time: 3.6870s
Epoch 55| Loss: 0.0587| Acc: 0.0517| Train Time: 2.8412s
Epoch 56| Loss: 0.0631| Acc: 0.0513| Train Time: 2.7303s
Epoch 57| Loss: 0.0508| Acc: 0.0517| Train Time: 3.2483s
Epoch 58| Loss: 0.0736| Acc: 0.0513| Train Time: 2.5251s
Epoch 59| Loss: 0.0726| Acc: 0.0513| Train Time: 3.5388s
Epoch 60| Loss: 0.0495| Acc: 0.0517| Train Time: 2.7057s


Evaluating: 100%|██████████| 8124/8124 [00:08<00:00, 918.34it/s]


Train: 0.9929| Val: 0.7740| Test: 0.7840| Eval Time: 8.8513s
Epoch 61| Loss: 0.0641| Acc: 0.0517| Train Time: 2.6360s
Epoch 62| Loss: 0.0654| Acc: 0.0517| Train Time: 2.5392s
Epoch 63| Loss: 0.0280| Acc: 0.0517| Train Time: 2.5542s
Epoch 64| Loss: 0.0409| Acc: 0.0513| Train Time: 2.5478s
Epoch 65| Loss: 0.0403| Acc: 0.0517| Train Time: 3.2255s
Epoch 66| Loss: 0.0457| Acc: 0.0517| Train Time: 2.5688s
Epoch 67| Loss: 0.0207| Acc: 0.0517| Train Time: 2.4004s
Epoch 68| Loss: 0.0491| Acc: 0.0517| Train Time: 2.2518s
Epoch 69| Loss: 0.0312| Acc: 0.0517| Train Time: 2.2016s
Epoch 70| Loss: 0.0304| Acc: 0.0513| Train Time: 2.3372s


Evaluating: 100%|██████████| 8124/8124 [00:08<00:00, 976.73it/s]


Train: 0.9929| Val: 0.7740| Test: 0.7820| Eval Time: 8.3256s
Epoch 71| Loss: 0.0376| Acc: 0.0517| Train Time: 2.7586s
Epoch 72| Loss: 0.0213| Acc: 0.0517| Train Time: 2.7666s
Epoch 73| Loss: 0.0120| Acc: 0.0517| Train Time: 2.3503s
